In [1]:
import numpy as np
from networkx.algorithms.tournament import hamiltonian_path

# 假设有 3 台机器和 3 个操作
m = 3  # 机器数量
N = 2  # 表示机器的二进制 (0, 1)
n = 2  # 操作数量
l = 1  # 用于表示 C_max 的二进制位数

# 处理时间矩阵 (例如)
p = np.array([[3, 1, 2], [3, 2, 1]])

b = np.array([[0, 0], [1, 0], [0, 1]])
# 非法数组(用于表示机器数)(1, 1)
bw = np.array([[1, 1]])
# 常数
K1 = 100
P = 30
# print(len(bw))
# 量子比特数量
num_qubits = n * N + l

In [2]:
#
# def pauli_z_term(index, num_qubits):
#     """初始化一个 Pauli-I 门的列表,指定索引 index 替换为 Pauli-Z 门"""
#     paulis = ['I'] * num_qubits
#     paulis[index] = 'Z'
#     return ''.join(paulis)  # 将列表转化为字符串
def pauli_z_term(indices, num_qubits):
    """生成 Pauli-Z 项"""
    z_str = ['I'] * num_qubits
    for idx in indices:
        z_str[idx] = 'Z'
    return ''.join(z_str)


def pauli_x_term(index, num_qubits):
    paulis = ['I'] * num_qubits
    paulis[index] = 'X'
    return ''.join(paulis)

In [3]:
from qiskit.quantum_info import Pauli, SparsePauliOp, PauliList

# 初始化 Pauli 字符串和系数
pauli_strings = []
coefficients = []

# 构建哈密顿量
for h in range(l):
    coef = - 2 ** h / 2  # 系数为 2^h
    z_term = ['I'] * num_qubits
    z_term[n * N + h] = 'Z'  # 第 n * N + h 位是 Z，其余是 I
    pauli_strings.append(''.join(z_term))
    coefficients.append(coef)

# 转换为 SparsePauliOp 格式
hamiltonian1 = SparsePauliOp.from_list(list(zip(pauli_strings, coefficients)))

# 输出结果
print("哈密顿量:")
print(hamiltonian1)

哈密顿量:
SparsePauliOp(['IIIIZ'],
              coeffs=[-0.5+0.j])


In [4]:
from itertools import combinations

# 初始化哈密顿量参数
pauli_strings = []
coefficients = []

# 遍历 j 和 i
for j in range(len(bw)):
    for i in range(n):
        # 枚举 k 的所有子集
        for subset_size in range(N + 1):  # 子集大小
            for subset in combinations(range(N), subset_size):
                # 计算子集 S 的系数
                coef = K1
                for k in subset:
                    coef *= (0.5 - b[j][k])  # 子集 S 中的项
                for k in range(N):
                    if k not in subset:
                        coef *= 0.5  # 非子集项的贡献

                # 生成对应的 Pauli-Z 项
                indices = [i * N + k for k in subset]  # 转为量子比特索引
                z_term = pauli_z_term(indices, num_qubits)

                # 添加到哈密顿量
                pauli_strings.append(z_term)
                coefficients.append(coef)

# 转换为 SparsePauliOp 格式
hamiltonian2 = SparsePauliOp.from_list(list(zip(pauli_strings, coefficients)))

# 输出结果
print("哈密顿量:")
print(hamiltonian2)

哈密顿量:
SparsePauliOp(['IIIII', 'ZIIII', 'IZIII', 'ZZIII', 'IIIII', 'IIZII', 'IIIZI', 'IIZZI'],
              coeffs=[25.+0.j, 25.+0.j, 25.+0.j, 25.+0.j, 25.+0.j, 25.+0.j, 25.+0.j, 25.+0.j])


In [5]:
# 初始化参数
pauli_strings = []
coefficients = []

# 第一部分：双重乘积项
for j in range(m):
    for i in range(n):
        for i_prime in range(n):
            coef = p[i, j] * p[i_prime, j] / (2 ** (2 * N))
            z_term = ['I'] * num_qubits
            for k in range(N):
                if b[j, k] == 1:
                    z_term[i * N + k] = 'Z'
                    z_term[i_prime * N + k] = 'Z'
            pauli_strings.append(''.join(z_term))
            coefficients.append(coef)

# 第二部分：结合时间和二进制位
for j in range(m):
    for i in range(n):
        for h in range(l):
            coef = -2 * p[i, j] * (2 ** (h - 1)) / (2 ** N)
            z_term = ['I'] * num_qubits
            for k in range(N):
                if b[j, k] == 1:
                    z_term[i * N + k] = 'Z'
            z_term[n * N + h] = 'Z'
            pauli_strings.append(''.join(z_term))
            coefficients.append(coef)

# 第三部分：纯二进制项
for h in range(l):
    for h_prime in range(l):
        coef = 2 ** (h - 1) * 2 ** (h_prime - 1)
        z_term = ['I'] * num_qubits
        if h == h_prime:
            z_term[n * N + h] = 'Z'
            pauli_strings.append(''.join(z_term))
            coefficients.append(coef)
        else:
            z_term1 = z_term.copy()
            z_term2 = z_term.copy()
            z_term1[n * N + h] = 'Z'
            z_term2[n * N + h_prime] = 'Z'
            pauli_strings.append(''.join(z_term1))
            coefficients.append(-coef)
            pauli_strings.append(''.join(z_term2))
            coefficients.append(-coef)
            z_term1[n * N + h_prime] = 'Z'
            pauli_strings.append(''.join(z_term1))
            coefficients.append(coef)

# 转换为 SparsePauliOp 格式
hamiltonian3 = SparsePauliOp.from_list(list(zip(pauli_strings, coefficients)))

# 输出结果
print("哈密顿量:")
print(hamiltonian3)

哈密顿量:
SparsePauliOp(['IIIII', 'IIIII', 'IIIII', 'IIIII', 'ZIIII', 'ZIZII', 'ZIZII', 'IIZII', 'IZIII', 'IZIZI', 'IZIZI', 'IIIZI', 'IIIIZ', 'IIIIZ', 'ZIIIZ', 'IIZIZ', 'IZIIZ', 'IIIZZ', 'IIIIZ'],
              coeffs=[ 0.5625+0.j,  0.5625+0.j,  0.5625+0.j,  0.5625+0.j,  0.0625+0.j,
  0.125 +0.j,  0.125 +0.j,  0.25  +0.j,  0.25  +0.j,  0.125 +0.j,
  0.125 +0.j,  0.0625+0.j, -0.75  +0.j, -0.75  +0.j, -0.25  +0.j,
 -0.5   +0.j, -0.5   +0.j, -0.25  +0.j,  0.25  +0.j])


In [6]:
cost_hamiltonian = hamiltonian1 + hamiltonian2 + hamiltonian3

In [7]:
def invert_counts(s):
    return s[::-1]

In [8]:
# # 初始化数组
# pauli_strings = []
# coefficients = []
# x_terms = []
# z_terms = []
# for i in range(num_qubits):
#     coef = 1
#     x_term = pauli_x_term(i,num_qubits)
#     x_terms.append((coef, x_term))
# 
# # 提取所有的 Pauli 字符串和系数
# pauli_strings = [term[1] for term in  x_terms ]
# coefficients = [term[0] for term in  x_terms ]
# 
# # 转换为 SparsePauliOp
# paulis = [Pauli(label) for label in pauli_strings]
# mixer_hamiltonian = SparsePauliOp.from_list(list(zip(pauli_strings, coefficients)))
# 
# print(mixer_hamiltonian)
# 算符池
operator_pool = [
    "X",  # X 门
    "Y",  # Y 门
    "XX",  # XX 门
    "YY",  # YY 门
    "ZZ",  # ZZ 门
    "XZ",  # XZ 门
    "YZ"  # YZ 门
]

In [9]:
# 定义混合算符池
mixer_pool = [
    SparsePauliOp.from_list([("X" * num_qubits, 1.0)]),
    SparsePauliOp.from_list([("Y" * num_qubits, 1.0)]),
    SparsePauliOp.from_list([("XX" * num_qubits, 1.0)]),
    SparsePauliOp.from_list([("YY" * num_qubits, 1.0)]),
    SparsePauliOp.from_list([("ZZ" * num_qubits, 1.0)]),
    SparsePauliOp.from_list([("XZ" * num_qubits, 1.0)]),
    SparsePauliOp.from_list([("YZ" * num_qubits, 1.0)])
]

In [10]:
def decompose_commutator(hamiltonian, operator):
    """
    分解 [H_C, A_j] 为 Pauli 算符的线性组合
    """
    commutator = hamiltonian @ operator - operator @ hamiltonian
    return commutator  # 返回 SparsePauliOp 表示

In [11]:
from qiskit.opflow import PauliSumOp, PauliTrotterEvolution


#from qiskit.opflow import PauliTrotterEvolution, PauliSumOp


def compute_gradient_via_commutator(hamiltonian, operator, gamma):
    """
    使用分解的对易算符计算梯度
    """
    # 1. 分解对易算符
    commutator = decompose_commutator(hamiltonian, operator)  # 对易算符分解
    pauli_list = commutator.to_list()  # 提取 Pauli 字符串及其系数

    # 2. 构建量子电路
    qc = QuantumCircuit(num_qubits, num_qubits)
    qc.h(range(num_qubits))  # 全局初态 |+>
    # qc.append(PauliEvolutionGate(hamiltonian, gamma), list(range(num_qubits)))
    # 转换为 PauliSumOp
    # 使用 PauliTrotterEvolution 创建演化电路
    hamiltonian = PauliSumOp(hamiltonian)
    evolution = PauliTrotterEvolution().convert((gamma * hamiltonian).exp_i())
    evolution_circuit = evolution.to_circuit()
    qc = qc.compose(evolution_circuit, qubits=range(num_qubits))  # 确保电路合成正确
    # 3. 使用 Aer 模拟器运行电路
    simulator = Aer.get_backend("aer_simulator")
    simulator.set_options(method="statevector")
    # 将量子电路分解为基础门
    qc_transpile = transpile(qc, simulator)
    # 运行分解后的电路
    # 配置运行任务返回 statevector
    result = simulator.run(qc_transpile).result()
    # 获取状态矢量
    statevector = result.get_statevector()

    # 4. 计算梯度
    gradient = 0.0
    for pauli_string, coeff in pauli_list:
        pauli_op = SparsePauliOp.from_list([(pauli_string, 1.0)])
        # 将 pauli_op 转换为稀疏矩阵
        pauli_matrix = pauli_op.to_matrix()
        expectation = np.vdot(statevector, pauli_matrix @ statevector)  # 期望值
        gradient += coeff * expectation  # 梯度累加

    return -1j * gradient.imag


C:\Users\李华宪\AppData\Local\Temp\ipykernel_42048\411534091.py:1: DeprecationWarning: The ``qiskit.opflow`` module is deprecated as of qiskit-terra 0.24.0. It will be removed in Qiskit 1.0. For code migration guidelines, visit https://qisk.it/opflow_migration.
  from qiskit.opflow import PauliSumOp, PauliTrotterEvolution


In [12]:
from qiskit.circuit.library import PauliEvolutionGate
from qiskit import Aer, transpile, assemble, QuantumCircuit, execute


# 定义成本函数
def cost_function(hamiltonian, params):
    beta, gamma = params[:len(params) // 2], params[len(params) // 2:]
    num_qubits = n * N + l
    qc = QuantumCircuit(num_qubits, num_qubits)
    qc.h(range(num_qubits))
    # print(qc)

    length = len(params)
    max_gradient = 0.0
    best_operator = None

    for operator in mixer_pool:
        # 比较绝对值找到最大梯度
        grad = compute_gradient_via_commutator(hamiltonian, operator, gamma[len(gamma) - 1])
        if abs(grad) > abs(max_gradient):
            max_gradient = grad
            best_operator = operator
    print("第", length / 2, "层：")
    print(f"\nBest Operator: {best_operator}, Max Gradient: {max_gradient}")
    for ga, be in zip(gamma, beta):
        # 应用问题哈密顿量
        qc.append(PauliEvolutionGate(hamiltonian, ga), list(range(num_qubits)))
        # 应用混合哈密顿量
        qc.append(PauliEvolutionGate(best_operator, be), list(range(num_qubits)))
    # # 计算梯度
    # grad = compute_gradient_via_commutator(qc, cost_hamiltonian, operator, gamma)
    # 模拟电路
    # 添加测量指令
    qc.measure(range(num_qubits), range(num_qubits))
    backend = Aer.get_backend('qasm_simulator')
    result = execute(qc, backend, seed_simulator=10, shots=10000).result()
    counts = result.get_counts(qc)
    # 计算期望值
    expectation = 0
    for bitstring, count in counts.items():
        bitstring = invert_counts(bitstring)
        prob = count / 10000
        z_h = np.array([int(bit) for bit in bitstring[(n * N):]])
        s_ik = np.array([int(bit) for bit in bitstring[:(n * N)]]).reshape(n, N)
        E1 = np.sum([2 ** h * z for h, z in enumerate(z_h)])
        E2 = 0
        for j in range(len(bw)):
            for i in range(n):
                constant = 1
                for k in range(N):
                    constant = constant * (1 - (s_ik[i, k] - bw[j, k]) ** 2)
                E2 += constant
        E2 = K1 * E2
        E3 = 0
        for j in range(len(b)):
            constant1 = 0
            for i in range(n):
                constant2 = 1
                for k in range(N):
                    constant2 = constant2 * (1 - (s_ik[i, k] - b[j, k]) ** 2)
                constant1 += p[i, j] * constant2
            constant3 = 0
            for h in range(l):
                constant3 += 2 ** h * z_h[h]
            constant = max(0, constant1 - constant3)
            E3 += P * (constant) ** 2

        # alpha_s = np.max([0, np.sum(x_ij * p) - np.sum([2 ** h * z for h, z in enumerate(z_h)])])

        E = E1 + E2 + E3

        expectation += E * prob

    return expectation, counts

In [13]:
from qiskit_algorithms.optimizers import COBYLA


# 优化 QAOA 参数
def optimize_qaoa(hamiltonian, params):
    def objective(params):
        expectation, _, grade = cost_function(hamiltonian, params)

        return expectation

    optimizer = COBYLA(rhobeg=1.5, tol=1e-8)
    # params = np.random.rand(2 * p_max) * 2 * np.pi
    result = optimizer.minimize(fun=objective, x0=params)

    return result

In [14]:
# def append_multi_rzz_term(qc, qubits, gamma): 
#     """
#     构建一个类RZZ门，n-1个控制位，1个目标位，进行旋转操作。
#     参数:
#     control_qubits: List[int]，控制量子比特的索引
#     target_qubit: int，目标量子比特的索引
#     """
#     length = len(qubits)
#     
#     if length==1:
#         qc.rz(gamma, qubits[0])
#     else:
#         control_qubits = list(qubits[:length-1])
# 
#         target_qubit = qubits[length-1]
# 
#         n_controls = len(control_qubits)
#         
#         if n_controls == 1:     
#             qc.cx(control_qubits,target_qubit)
#         else:
#             qc.mcx(control_qubits, target_qubit)
#         qc.rz(gamma, target_qubit)
#         if n_controls == 1:
#             qc.cx(control_qubits, target_qubit)
#         else:
#             qc.mcx(control_qubits, target_qubit)


In [15]:
# def append__zz_term( qc, qubit1, qubit2, gamma):
#     qc.cx(qubit1, qubit2)
#     qc.rz(gamma, qubit2)
#     qc.cx(qubit1, qubit2)
# 
# def append__z_term(qc, qubit, gamma):
#     qc.rz(gamma, qubit)
# def append__mixer_term(qc, qubit, beta):
#     qc.rx(2 * beta, qubit)

\begin{align}\sum_{h=0}^{l} 2^{h-1} \end{align}

In [16]:
# def get_cost_circuit1(gamma, qc):
#     for h in range(l):
#         coef = - 2**h
#         append__z_term(qc, N * n + h, coef * gamma)
#     return qc

\begin{align}K_{1} \sum_{j\in W} \sum_{i=0}^{n}\prod_{k=0}^{N}\left(\frac{1}2+\left(\frac{1}2-b_{j k}\right)Z_{ik}\right)\end{align}

In [17]:
# import itertools
# def get_cost_circuit2(gamma, qc):
#     # print(len(bw))
#     if len(bw) == 0:
#         return qc
#     all_combinations = list(itertools.chain(*(itertools.combinations(range(N), r) for r in range(1, N + 1))))
#     for j in range(len(bw)):
#         for i in range(n):
#             constant = 1 / (2 ** N)  # 系数 1/2^N  
#             for qubit_indices in all_combinations: # 遍历各种组合
#                 qubits_index = np.array([], dtype=int)
#                 # 根据组合项生成子项的常数和qubit索引
#                 for k in range(N):
#                     constant *= (1 - 2 * bw[j, k]) if k in qubit_indices else 1
#                     if k in qubit_indices:
#                         qubits_index = np.append(qubits_index, i * N + k)
#                 # print("qubits_index", qubits_index)
#                 append_multi_rzz_term(qc, qubits_index, 2 * K1 * constant * gamma)
#     return qc


In [18]:
# from qiskit import Aer, QuantumCircuit, execute
# gamma= 1
# qc = QuantumCircuit(num_qubits, num_qubits)
# # qc.h(range(num_qubits))
# # print(qc)
# get_cost_circuit2(gamma, qc)
# print(qc)
# # qc.decompose().draw('mpl')

\begin{align}\sum_{j=0}^{m-1}\sum_{i=0}^{n-1} \sum_{i^{\prime}=0}^{n-1}\left(\prod_{k=0}^{N-1}\frac{1}{2}\left(1+\left(1-2b_{j k}\right) Z_{i k}\right)\right) p_{i j}\left(\prod_{k^{\prime}=0}^{N-1}\frac{1}{2}\left(1+\left(1-2b_{j k}\right) Z_{i^{\prime} k^{\prime}}\right)\right) p_{i^{\prime} j}\end{align}
\begin{align}-2 \sum_{j=0}^{m-1} \sum_{i=0}^{n-1} \sum_{h=0}^{l-1} \left(\prod_{k=0}^{N-1}\frac{1}{2}\left(1+\left(1-2b_{j k}\right) Z_{i k}\right)\right) p_{i j} \cdot 2^{h-1}\left(1-Z_h\right)\end{align}
\begin{align}\sum_{j=0}^{m-1}\sum_{h=0}^{l-1} \sum_{h^{\prime}=0}^{l-1} 2^{h-1} 2^{h^{\prime}-1}\left(1-Z_h-Z_{h^{\prime}}+Z_h Z_{h^{\prime}}\right)\end{align}

In [19]:
# # import itertools
# from collections import Counter
# # 预生成所有比特组合
# # all_combinations = list(itertools.chain(*(itertools.combinations(range(N), r) for r in range(1, N + 1))))
# def get_cost_circuit3(gamma, qc):
#     for j in range(len(b)):
#         for i in range(n): # 自身内部相乘
#             all_combinations_1 = list(itertools.chain(*(itertools.combinations(range(N), r) for r in range(1, N + 1))))
#             # 获取 all_combinations_1 中任意两项的所有组合
#             all_two_combinations = list(itertools.combinations(all_combinations_1, 2))
#             for qubit_indices in all_two_combinations: # 遍历各种组合
#                 # print("qubit_indices", qubit_indices)
#                 # sub_constant = constant
#                 qubits_index = np.array([], dtype=int)
#                 constant = 1
#                 # 将组合对中的每个元素（每个元素本身也是一个组合）放入数组
#                 for combo in qubit_indices: # qubit_indices: ((0,), (1,)) 
#                     for k in combo: # combo: (0,)
#                         constant = constant * (1 - 2 * b[j, k])
#                         qubits_index = np.append(qubits_index, i * N + k)
#                     # 将组合项中的所有元素添加到 qubits_index 中
#                 # 剔除出现偶次数的项，奇次数的项保留一个
#                 # print("qubits_index", qubits_index)
#                 count = Counter(qubits_index)
#                 # 仅保留出现次数为奇数的元素一次   
#                 qubits_index = [x for x in count if count[x] % 2 != 0]
#                 # print("qubits_index", qubits_index)
#                 constant = 1/(2 ** (2 * N)) * constant * p[i, j] * p[i, j]
#                 append_multi_rzz_term(qc,qubits_index , 2 * P * constant * gamma)
#         # 2，对应两两相乘
#         all_combinations_1 = list(itertools.chain(*(itertools.combinations(range(N), r) for r in range(1, N + 1))))  
#         all_combinations_2 = list(itertools.chain(*(itertools.combinations(range(N), r) for r in range(1, N + 1))))  
#         for i1 in range(n - 1): 
#             for i2 in range(i1 + 1, n): 
#                 for qubit_indices_1 in all_combinations_1:
#                     # print("qubit_indices1", qubit_indices_1)
#                     for qubit_indices_2 in all_combinations_2:
#                         # print("qubit_indices2", qubit_indices_2)
#                         qubits_index = np.array([], dtype=int)
#                         constant = 1
#                         for k1 in qubit_indices_1: # qubit_indices: ((0,), (1,))
#                             constant = constant * (1 - 2 * b[j, k1])
#                             qubits_index = np.append(qubits_index, i1 * N + k1)
#                         for k2 in qubit_indices_2: # qubit_indices: ((0,), (1,))
#                             constant = constant * (1 - 2 * b[j, k2])
#                             qubits_index = np.append(qubits_index, i2 * N + k2)
#                         constant = 1/(2 ** (2 * N)) * constant * p[i1, j] * p[i2, j]
#                         # print("qubits_index", qubits_index)
#                         append_multi_rzz_term(qc,qubits_index ,2 * P * constant * gamma)
#         # 对应常数相乘
#         all_combinations = list(itertools.chain(*(itertools.combinations(range(N), r) for r in range(1, N + 1))))
#         for i1 in range(n): # 取常数 1 固定一个为 p[i_1, j]
#             for i2 in range(n): # 取 Z 的下标 Z_i2k
#                 for qubit_indices in all_combinations: # 取 排列组合
#                     qubits_index = np.array([], dtype=int)
#                     constant = 1
#                     for k in qubit_indices:
#                         constant = constant * (1 - 2 * b[j, k])
#                         qubits_index = np.append(qubits_index, i2 * N + k)
#                     constant = 1/(2 ** (2 * N)) * constant * p[i1, j] * p[i2, j]
#                     # print("i1:",i1,"i2:",i2,)
#                     # print("qubits_index", qubits_index)
#                     append_multi_rzz_term(qc,qubits_index ,2 * P * constant * gamma)
#         all_combinations = list(itertools.chain(*(itertools.combinations(range(N), r) for r in range(1, N + 1))))
#         for i in range(n):
#             constant = 1 / (2 ** N) * p[i, j]# 系数 
# 
#             for qubit_indices in all_combinations: # 遍历各种组合
#                 constant1 = constant
#                 qubits_index = np.array([], dtype=int)
#                 # 根据组合项生成子项的常数和qubit索引
#                 for k in range(N):
#                     constant1 *= (1 - 2 * b[j, k]) if k in qubit_indices else 1
#                     if k in qubit_indices:
#                         qubits_index = np.append(qubits_index, i * N + k)            
#                 constant1 = constant1 * (sum(2**(h - 1) for h in range(l))) * (-2)
#                 append_multi_rzz_term(qc, qubits_index, 2 * P * constant1 * gamma)
# 
#             for h in range(l):
#                 for qubit_indices in all_combinations: # 遍历各种组合
#                     constant2 = constant
#                     qubits_index = np.array([], dtype=int)
#                     # 根据组合项生成子项的常数和qubit索引
#                     for k in range(N):
#                         constant2 *= (1 - 2 * b[j, k]) if k in qubit_indices else 1
#                         if k in qubit_indices:
#                             qubits_index = np.append(qubits_index, i * N + k)
# 
#                     qubits_index = np.append(qubits_index, n * N + h)
#                     # print("qubits_index_hz", qubits_index)
#                     constant2 = constant2 * (2**(h-1)) * 2
#                     append_multi_rzz_term(qc, qubits_index, 2 * P * constant2 * gamma)
# 
#             for h in range(l):
#                 constant3 = constant
#                 qubits_index = np.array([], dtype=int)
#                 # 根据组合项生成子项的常数和qubit索引
#                 qubits_index = np.append(qubits_index, n * N + h)
#                 # print("qubits_index_hz", qubits_index)
#                 constant3 = constant3 * (2**(h-1)) * 2
#                 append_multi_rzz_term(qc, qubits_index, 2 * P * constant3 * gamma)
# 
# 
#         for h1 in range(l):  # 遍历 h 的范围
#             for h2 in range(l):  # 遍历 h' 的范围
#                 coef = 2 ** (h1 - 1) * 2 ** (h2 - 1)  # 计算系数权重
# 
#                 # -Z_h1 项
#                 append__z_term(qc, n * N + h1, 2 * P * coef * gamma)
# 
#                 # -Z_h2 项
#                 append__z_term(qc, n * N + h2, 2 * P * coef * gamma)
# 
#                 # Z_h1 Z_h2 项
#                 if h1 != h2:  # 避免重复
#                     append__zz_term(qc, n * N + h1, n * N + h2, 2 * P * coef * gamma)




In [20]:
# from qiskit import Aer, QuantumCircuit, execute
# gamma= 1
# qc = QuantumCircuit(num_qubits, num_qubits)
# get_cost_circuit3(gamma, qc)
# # print(qc)
# # qc.decompose().draw('mpl')

In [56]:
# import numpy as np
# from qiskit import QuantumCircuit, Aer, transpile
# from qiskit.circuit.library import RXGate, RYGate, RZZGate, RZXGate, RXXGate, RYYGate, RZZGate, RZGate
# 
# # 添加单层算符
# def add_layer(circuit, operator, beta):
#     if operator == "X":
#         for qubit in range(num_qubits):
#             circuit.rx(2 * beta, qubit)
#     elif operator == "Y":
#         for qubit in range(num_qubits):
#             circuit.ry(2 * beta, qubit)
#     elif operator == "XX":
#         for i in range(num_qubits - 1):
#             circuit.append(RXXGate(2 * beta), [i, i + 1])
#     elif operator == "YY":
#         for i in range(num_qubits - 1):
#             circuit.append(RYYGate(2 * beta), [i, i + 1])
#     elif operator == "ZZ":
#         for i in range(num_qubits - 1):
#             circuit.append(RZZGate(2 * beta), [i, i + 1])
#     elif operator == "XZ":
#         for i in range(num_qubits - 1):
#             circuit.rx(2 * beta, i)
#             circuit.rz(2 * beta, i + 1)
#     elif operator == "YZ":
#         for i in range(num_qubits - 1):
#             circuit.ry(2 * beta, i)
#             circuit.rz(2 * beta, i + 1)


In [57]:
# def get_mixer_circuit(beta, qc):
#     # beta, gamma = params[:len(params)//2], params[len(params)//2:]
#     for i in range(num_qubits):
#         append__mixer_term(qc, i, 2 * beta)
#     return qc

In [58]:
# def invert_counts(s):
#     return s[::-1]

In [59]:
# from qiskit import Aer, QuantumCircuit, execute
# # 定义成本函数
# def cost_function(params, mix_operator):
#     beta, gamma = params[:len(params)//2], params[len(params)//2:]
#     num_qubits = n * N + l
#     qc = QuantumCircuit(num_qubits, num_qubits)
#     qc.h(range(num_qubits))
#     # print(qc)
#     # operator_pool = build_operator_pool(num_qubits)
#     # operator = np.random.choice(operator_pool)
#     
#     length  = len(params)
#     for i in range(int(length/2)):
# 
#         get_cost_circuit1(gamma[i], qc)
#         get_cost_circuit2(gamma[i], qc)
#         get_cost_circuit3(gamma[i], qc)
#         add_layer(qc, mix_operator, beta[i])
#         # get_mixer_circuit(beta[i], qc)
#     # 模拟电路
#     # 添加测量指令
#     qc.measure(range(num_qubits), range(num_qubits))
#     backend = Aer.get_backend('qasm_simulator')
#     result = execute(qc, backend, seed_simulator=10, shots=10000).result()
#     counts = result.get_counts(qc)
#     # 计算期望值
#     expectation = 0
#     for bitstring, count in counts.items():
#         bitstring = invert_counts(bitstring)
#         prob = count / 10000
#         z_h = np.array([int(bit) for bit in bitstring[(n*N):]])
#         s_ik = np.array([int(bit) for bit in bitstring[:(n*N)]]).reshape(n, N)
#         E1 = np.sum([2 ** h * z for h, z in enumerate(z_h)])
#         E2 = 0
#         for j in range(len(bw)):
#             for i in range (n):
#                 constant = 1
#                 for k in range (N):
#                     constant = constant*(1-(s_ik[i,k]-bw[j,k])**2)
#                 E2 += constant
#         E2 = K1 * E2
#         E3 = 0
#         for j in range(m):
#             constant1 = 0
#             for i in range(n):
#                 constant2 = 1
#                 for k in range(N):
#                     constant2 = constant2*(1-(s_ik[i,k]-b[j,k])**2)
#                 constant1 += p[i,j]*constant2
#             constant3 = 0
#             for h in range(l):
#                 constant3 += 2 ** h * z_h[h]
#             constant = max(0, constant1 - constant3)
#             E3 += P * (constant)**2
#         E =  E1 + E2 + E3
#   
#         expectation += E * prob
# 
#     return expectation, counts

In [60]:
# from qiskit_algorithms.optimizers import COBYLA
# 
# # 优化 QAOA 参数
# def optimize_qaoa(params, mix_operator):
#     def objective(params):
#         expectation, _ = cost_function(params, mix_operator)
# 
#         return expectation
# 
#     optimizer = COBYLA(rhobeg = 1.5, tol = 1e-8)
#     # params = np.random.rand(2 * p_max) * 2 * np.pi
#     result = optimizer.minimize(fun=objective, x0=params)
# 
#     return result

In [61]:
import time
from qiskit.visualization import plot_histogram

# 算符池
# operator_pool = [
#     "X",        # X 门 
#     "Y",        # Y 门
#     "XX",       # XX 门
#     "YY",       # YY 门
#     "ZZ",       # ZZ 门
#     "XZ",       # XZ 门
#     "YZ"        # YZ 门
# ]
# 运行优化

startTime = time.time()
ansatz_final = object()
init_point = np.array([])
counts = {}
min_energy = 0
hamiltonian = cost_hamiltonian
final_result = object()

for p1 in range(1, 2):
    min_energy = 10000
    for k in range(5):
        # print('第：', p1, '层', ' ', '第', k, '个参数')
        init_point_temp = init_point
        beta_p = np.random.uniform(0, np.pi)
        gamma_p = np.random.uniform(0, 2 * np.pi)
        if k > 0:
            init_point_temp = np.delete(init_point_temp, 2 * p1 - 1)
            init_point_temp = np.delete(init_point_temp, p1 - 1)
        init_point_temp = np.insert(init_point_temp, p1 - 1, beta_p)
        init_point_temp = np.insert(init_point_temp, 2 * p1 - 1, gamma_p)
        result = optimize_qaoa(hamiltonian, init_point_temp)

        # print(result.x)
        optimal_params = result.x

        # 计算最终的能量
        energy, counts_temp = cost_function(hamiltonian, result.x)
        if min_energy > energy:
            min_energy = energy
            init_point = optimal_params
            counts = counts_temp
            final_result = result
endTime = time.time()

# 输出结果
print("优化时间：", endTime - startTime)
print("Optimal parameters:", final_result.x)
print("Optimal value:", final_result.fun)
# 使用最优参数运行量子电路并输出测量结果
final_expectation, final_counts = cost_function(hamiltonian, final_result.x)
sorted_dict = {item[0][::-1]: item[1] for item in sorted(final_counts.items(), key=lambda item: item[1], reverse=True)}
print("Final expectation value:", final_expectation)
print("Final measurement counts:", sorted_dict)
plot_histogram(sorted_dict)

C:\Users\李华宪\AppData\Local\Temp\ipykernel_21316\411534091.py:21: DeprecationWarning: The class ``qiskit.opflow.primitive_ops.pauli_sum_op.PauliSumOp`` is deprecated as of qiskit-terra 0.24.0. It will be removed no earlier than 3 months after the release date. For code migration guidelines, visit https://qisk.it/opflow_migration.
  hamiltonian = PauliSumOp(hamiltonian)
C:\Users\李华宪\AppData\Local\Temp\ipykernel_21316\411534091.py:22: DeprecationWarning: The class ``qiskit.opflow.evolutions.pauli_trotter_evolution.PauliTrotterEvolution`` is deprecated as of qiskit-terra 0.24.0. It will be removed no earlier than 3 months after the release date. For code migration guidelines, visit https://qisk.it/opflow_migration.
  evolution = PauliTrotterEvolution().convert((gamma * hamiltonian).exp_i())


ValueError: setting an array element with a sequence. The requested array would exceed the maximum number of dimension of 32.

In [93]:
# from qiskit.visualization import plot_histogram
# 
# # 输出结果
# print("优化时间：", endTime - startTime)
# print("Optimal parameters:", final_result.x)
# print("Optimal value:", final_result.fun)
# # 使用最优参数运行量子电路并输出测量结果
# final_expectation, final_counts = cost_function(final_result.x)
# sorted_dict = {item[0][::-1]: item[1] for item in sorted(final_counts.items(), key=lambda item: item[1], reverse=True)}
# print("Final expectation value:", final_expectation)
# print("Final measurement counts:", sorted_dict)
# plot_histogram(sorted_dict)

In [ ]:
expectation = 0
bitstring = "11111"
z_h = np.array([int(bit) for bit in bitstring[(n * N):]])
s_ik = np.array([int(bit) for bit in bitstring[:(n * N)]]).reshape(n, N)
E1 = np.sum([2 ** h * z for h, z in enumerate(z_h)])
E2 = 0
for j in range(len(bw)):
    for i in range(n):
        constant = 1
        for k in range(N):
            constant = constant * (1 - (s_ik[i, k] - bw[j, k]) ** 2)
        E2 += constant
E2 = K1 * E2
E3 = 0
for j in range(m):
    constant1 = 0
    # print("机器：", j)
    for i in range(n):
        constant2 = 1
        for k in range(N):
            constant2 = constant2 * (1 - (s_ik[i, k] - b[j, k]) ** 2)
        # print("constant2:", constant2)
        # print("p[i,j]:", p[i,j])
        constant1 += p[i, j] * constant2
    # print("constant1:", constant1)
    constant3 = 0
    for h in range(l):
        constant3 += 2 ** h * z_h[h]
    # print("constant3:", constant3)
    constant = max(0, constant1 - constant3)
    E3 += (constant) ** 2

    # alpha_s = np.max([0, np.sum(x_ij * p) - np.sum([2 ** h * z for h, z in enumerate(z_h)])])

E = E1 + K1 * E2 + P * E3
print(E1)
print(E2)
print(E3)
print(E)
# print(e1)
# print(e2)